In [56]:
import ROOT as r
import math
import array
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, '../include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE

In [57]:
config="3"

fileDIR=(f"../outfiles/Pass1/QE_data_GEN{config}_sbs100p_nucleon_np_model2.root")
resolved_path = os.path.abspath(fileDIR)
print(f"Resolved Path: {resolved_path}")
print(f"File Exists: {os.path.exists(resolved_path)}")
def Function_ASYMMETRY(config):
    r.gStyle.SetOptStat(0)
    r.gStyle.SetOptFit(1)
    
    #_____bring in config values______#
    coinVector=CONFIG.Function_JSON("GEN"+config,"../config/coin.cfg")
    he3spot=CONFIG.Function_JSON("GEN"+config+"He3","../config/spotsize.cfg")
    he3spotsim=CONFIG.Function_JSON("GEN"+config+"He3sim","../config/spotsize.cfg")
    
    
    
    #here, set coincidence cut and spot cut values
    coinSigma=3
    coin_low=coinVector[0]-coinSigma*coinVector[1]
    coin_high=coinVector[0]+coinSigma*coinVector[1]

    hix_n_3=he3spotsim[7];
    lowx_n_3=he3spotsim[6];
    hix_p_3=he3spotsim[5];
    lowx_p_3=he3spotsim[4];
    hiy_p3=he3spotsim[1];
    lowy_p3=he3spotsim[0];
    hiy_n3=he3spotsim[3];
    lowy_n3=he3spotsim[2];
    
    
    hiydata=he3spot[1]+.75;
    lowydata=he3spot[0]-.75;
    hixdatan=he3spot[7];
    lowxdatan=he3spot[6];
    hixdatap=he3spot[5];
    lowxdatap=he3spot[4];


    
    
    
    
    
    
        # Define ROOT file paths
    rootfilenp = f"../outfiles/Pass1/QE_data_GEN{config}_sbs100p_nucleon_np_model2.root"
    rootfilep = f"../outfiles/Pass1/QE_sim_GEN{config}_sbs100p_nucleon_np_model2.root"

    if config == "4":
        config = "4b"

        # Load the TTrees
    B = r.TChain("Tout")
    C = r.TChain("Tout")

    dx_np = array.array('d', [0])
    dx_p = array.array('d', [0])
    dy_np = array.array('d', [0])
    dy_p = array.array('d', [0])
    W2_np = array.array('d', [0])
    W2_p = array.array('d', [0])
    coin_np = array.array('d', [0])
    weight = array.array('d', [0])
    helicity_p = array.array('i', [0])
    IHWP_p = array.array('i', [0])
    runnum_p = array.array('i', [0])
    helicity_np = array.array('i', [0])
    IHWP_np = array.array('i', [0])
    runnum_np = array.array('i', [0])

    # Load the TTrees

    C.Add(rootfilenp)
    B.Add(rootfilep)

    # Disable all branches initially
    C.SetBranchStatus("*", 0)
    B.SetBranchStatus("*", 0)

    # Enable specific branches
    branches = ["dx", "dy", "W2", "helicity", "IHWP", "runnum", "coinCut", "coin_time"]
    b2=["dx", "dy", "W2"]
    for branch in branches:
        C.SetBranchStatus(branch, 1)
    for branch in b2:
        B.SetBranchStatus(branch, 1)

    B.SetBranchStatus("weight", 1)

    # Set branch addresses
    C.SetBranchAddress("dx", dx_np)
    B.SetBranchAddress("dx", dx_p)
    C.SetBranchAddress("dy", dy_np)
    B.SetBranchAddress("dy", dy_p)
    C.SetBranchAddress("W2", W2_np)
    B.SetBranchAddress("W2", W2_p)
    C.SetBranchAddress("helicity", helicity_np)
    #B.SetBranchAddress("helicity", helicity_p)
    C.SetBranchAddress("IHWP", IHWP_np)
    #B.SetBranchAddress("IHWP", IHWP_p)
    C.SetBranchAddress("coin_time", coin_np)
    #B.SetBranchAddress("coin_time", coin_pp)
    C.SetBranchAddress("runnum", runnum_np)
    #B.SetBranchAddress("runnum", runnum_p)
    B.SetBranchAddress("weight", weight)
    # Assuming the variables are already defined or loaded from the ROOT file
    

    
    W2max=1.7
    W2min=-100
    if config=="4":
        W2max=2
        W2min=-1.2
        number=1.13
        hiydata=number
        lowydata=-number        

    
    nEntries_np = C.GetEntries()
    nEntries_p = B.GetEntries()
    C.GetEntry(0)
    thisnum = runnum_np[0]
    nplus_np = 0
    pplus_np = 0
    nminus_np = 0
    pminus_np = 0
    pYield=[]
    nYield=[]
    runnumVec = []
    runnumA_p = []
    runnumA_n = []
    runnumA_targetpol = []
    runnumA_beampol = []
    Err_A_n = []
    Err_A_p = []
    QE = 0
    analyze=True
    # Loop over the entries
    for i in range(nEntries_np):
        C.GetEntry(i)
        if runnum_np[0] > 2000:
            if thisnum == runnum_np[0]:
                if IHWP_np[0] == 1:
                    helicity_np[0] *= -1 * -1

                elif IHWP_np[0] == -1:
                    helicity_np[0] *= 1 * -1
                else:
                    continue
                
                ycut = lowydata < dy_np[0] < hiydata
                xcutn = lowxdatan < dx_np[0] < hixdatan
                xcutp = lowxdatap < dx_np[0] < hixdatap
                coin_cut_np = coin_low < coin_np[0] < coin_high and W2min < W2_np[0] < W2max
                
                if coin_cut_np and runnum_np[0] > 2165 and ycut and xcutn:
                    QE += 1

                    if helicity_np[0] == 1:
                        nplus_np += 1
                    if helicity_np[0] == -1:
                        nminus_np += 1
                
                if coin_cut_np and runnum_np[0] > 2165 and ycut and xcutp:
                    QE += 1
                    if helicity_np[0] == 1:
                        pplus_np += 1
                    if helicity_np[0] == -1:
                        pminus_np += 1
        
            else:
                analyze = True
                #print(f"{QE} Quasi elastic events.")
                if QE < 100:
                    analyze = False
                if analyze:
                    n_Asym = (nplus_np - nminus_np) * 1.0 / (nplus_np + nminus_np)
                    p_Asym = (pplus_np - pminus_np) * 1.0 / (pplus_np + pminus_np)
                    #print(f"Asymmetry for run number {thisnum}: {n_Asym} {p_Asym}")
                    if runnum_np[0] > 2165:
                        runnumVec.append(runnum_np[0])
                        runnumA_n.append(n_Asym)
                        runnumA_p.append(p_Asym)
                        pYield.append(pplus_np+pminus_np)
                        nYield.append(nplus_np+nminus_np)
                        Err_A_n.append(2 * math.sqrt(nplus_np * nminus_np) / (nplus_np + nminus_np))
                        Err_A_p.append(2 * math.sqrt(pplus_np * pminus_np) / (pplus_np + pminus_np))
                        thisnum = runnum_np[0]
                    QE = 0
                    nminus_np = 0
                    nplus_np = 0
            
                thisnum = runnum_np[0]
        else:
            thisnum = runnum_np[0]
    return runnumVec,runnumA_n,runnumA_p,Err_A_n,Err_A_p,nYield,pYield
   

Resolved Path: /home/research/GEn/GEn Data Analysis/outfiles/Pass1/QE_data_GEN3_sbs100p_nucleon_np_model2.root
File Exists: True


# Calculate the asymmetry and get the errors, yields, and polarizations by run number

In [24]:
runs,An,Ap,EAn,EAp,nYield,pYield=Function_ASYMMETRY("3")

In [8]:
import numpy as np
beamPol=np.empty(0);
he3Pol=np.empty(0);
for i in range(0,len(runs)):
    beamPol=np.append(beamPol,DBPARSE.Function_RETURNBEAMPOL(runs[i],"../DB/RunDates.csv","../DB/Beam_pol.csv"))
    he3Pol=np.append(he3Pol,DBPARSE.Function_RETURNHE3POL(runs[i],"../DB/RunDates.csv","../DB/He3_pol.csv"))

# Hear we compute the weighted average

In [30]:
newAn=np.array(An)
newAp=np.array(Ap)
newnYield=np.array(nYield)
newpYield=np.array(pYield)

In [47]:
weightedSumN=0
weightedSumP=0
sumWeightsN=0
sumWeightsP=0
for i in range(0,len(newAn)):
    weightedSumN+=newAn[i]*newnYield[i]*beamPol[i]*he3Pol[i]/100/100
    sumWeightsN+=newnYield[i]
    sumWeightsP+=newpYield[i]
    weightedSumP+=newAp[i]*newpYield[i]*beamPol[i]*he3Pol[i]/100/100
    
weighted_An=weightedSumN/sumWeightsN
weighted_Ap=weightedSumP/sumWeightsP

# Save Array Of Results

In [58]:
np.save(f'SavedArrays/GEN{config}/Pass1/GEN3Pass1.npy', {
    'An': newAn,
    'Ap': newAp,
    'EAn': EAn,
    'EAp':EAp,
    'nY':nYield,
    'pY':pYield,
    'beamPol':beamPol,
    'he3Pol':he3Pol,   
    'weightedAn':weighted_An,
    'weightedAp':weighted_Ap,
})

In [48]:
weighted_Ap*100

0.03439255172584925

In [49]:
weighted_An*100

-0.5427500416134725